In [1]:
import pandas as pd
import openai

In [2]:
df = pd.read_csv('news_multi_seed.csv')

In [3]:
from openai import OpenAI

client = OpenAI(api_key="")

In [4]:
def request_response_from_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        #prompt=prompt,
        messages=[
            {"role": "system", "content": "You are a crowdsourcing worker."},
            {"role": "user", "content": prompt}],
        n=1)
    return response

In [5]:
def request_with_checks(prompt):
    success = False
    count = 0
    while not success:
        if count > 0:
            print(f'Retrying with again. Current number of retries: {count}')
        if count >= 10:
            raise Exception('Too many attempts')
        try:
            response = request_response_from_gpt(prompt)
            sucess = True
            break
        except openai.error.RateLimitError as e:
            print(e)
            time.sleep(10)
            count += 1
        except openai.error.APIConnectionError as e:
            print(e)
            time.sleep(5)
            count += 1
        except openai.error.APIError or openai.error.JSONDecodeError as e:
            print(e)
            time.sleep(5)
            count += 1
        except openai.error.Timeout as e:
            print(e)
            time.sleep(5)
            count += 1
        except openai.error.ServiceUnavailableError as e:
            print(e)
            time.sleep(5)
            count += 1
    return response

In [6]:
def collect_samples(dct_final_prompts):
    dct_responses = {}

    for idx, key in enumerate(dct_final_prompts):
        print(str(idx))
        dct_responses[key] = []
        for no, prompt in enumerate(dct_final_prompts[key]):
            if no > 0 and no % 50 == 0:
                print(no)
            response = request_with_checks(prompt[0])
            dct_responses[key].append((response, prompt[1]))
            
    return dct_responses

In [7]:
dct_phrases = df.groupby('label')['text'].apply(set).to_dict()

In [9]:
default_prompt = 'Please provide 15 different changes of the Text by paraphrasing it. Output the full sentences. Output in format "1. sentence 1, 2. sentence 2, ... , 15. sentence 15". Text: "{}".'
    
dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append((default_prompt.format(phrase), phrase))

In [10]:
dct_responses = collect_samples(dct_final_prompts)

0
50
1
50
2
50
3
50


In [11]:
import re
import string

def filter_responses(dct_responses):
    dct_df = {'label': [], 'text': [], 'seed': []}
    for key in dct_responses:
        for responses in dct_responses[key]:
            for response in responses[0].choices:
                contents = response.message.content.split('\n')
                for content in contents:
                    if len(content) == 0:
                        continue
                    if content[0].isnumeric():
                        content = content[3:]
                    else:
                        continue
                    dct_df['label'].append(key)
                    dct_df['text'].append(content)
                    dct_df['seed'].append(responses[1])
                    
    fb_0 = pd.DataFrame.from_dict(dct_df)
                    
    fb_0['text']=fb_0['text'].apply(lambda x: x.replace('"',''))
    
    return fb_0

In [12]:
filter_responses(dct_responses).to_csv('news_gpt_para.csv', index=False)

In [13]:
default_prompt = 'Please provide 15 different changes of the Text by inserting words into the Text. Output the full sentences. Output in format "1. sentence 1, 2. sentence 2, ... , 15. sentence 15". Text: "{}".'
    
dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append((default_prompt.format(phrase), phrase))

In [15]:
dct_responses = collect_samples(dct_final_prompts)

0
50
1
50
2
50
3
50


In [16]:
filter_responses(dct_responses).to_csv('news_gpt_ins.csv', index=False)

In [17]:
default_prompt = 'Please provide 15 different changes of the Text by swapping words for their synonyms. Output the full sentences. Output in format "1. sentence 1, 2. sentence 2, ... , 15. sentence 15". Text: "{}".'

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append((default_prompt.format(phrase), phrase))

In [18]:
dct_responses = collect_samples(dct_final_prompts)

0
50
1
50
2
50
3
50


In [19]:
filter_responses(dct_responses).to_csv('news_gpt_swap.csv', index=False)